# Following trends on Twitter

[1. Abstract](#Abstract)

[2. Exploratory data analysis](#Exploratory-data-analysis)

[3. Topic Modeling using LDA](#A-very-simple-Topic-Modeling-using-LDA)

[4. Example of the pipeline](#Example-of-the-pipeline-that-we-will-follow-for-the-LDA-algorithm)

[5. Milestone 3](#Milestone-3:-the-data-story)

# Abstract


We realiazed that it will be hard to achieve the goal stated in milestone 1(detecting fake news). The problem is that we couldn't find a way to define fake news. And also the twitter dataset is not what we expected it to be. For example it doesn't contain the number of times a tweet has been retweeted, the geographical location, number of likes ... So we decided to go in a different, more feasable direction, which is following the process of creating and spreading trends on Twitter. Trying to find patterns between trends and users. 

In [1]:
import numpy as np
import json
import re
from pyspark.sql import *
from pyspark import SparkContext, SQLContext
from pyspark.ml.feature import *
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.sql import functions as F
import pickle

#### Read data

> Each data entry has 5 fields:
     - language: language of the user 
     - id: id of the user
     - date: date when the tweet was published
     - username: username of the user
     - content: the tweet
     
Given that for the moment we consider only rows that have all 5 fields we don't have to deal with missing values.
     

In [18]:
sqlContext = SQLContext(sc)

data = sc.textFile("/datasets/tweets-leon")
data

/datasets/tweets-leon MapPartitionsRDD[21] at textFile at NativeMethodAccessorImpl.java:-2

# Exploratory data analysis

> 1 We will first clean the data and select only a subset that is useful for this project:
    - keep only the tweets that have all 5 fields
    - remove urls from the content
    - remove emojis
    - remove punctuation 
    - remove stopwords
    - apply lemmatization 
    - keep only english, spanish and french tweets
    - ...

In [19]:
frist_tweet = data.first()
frist_tweet

Py4JJavaError: An error occurred while calling o269.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/datasets/tweets-leon
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:285)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:228)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:313)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:202)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:237)
	at scala.Option.getOrElse(Option.scala:120)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:237)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:237)
	at scala.Option.getOrElse(Optio

In [42]:
"""Chose tweets that have exactly 5 components like normal 
         (language, id, date, username, content)"""

def selection_tweet(tweet):
    return len(tweet.split("\t")) == 5

In [43]:
"encode tweet"

def encode_tweet(tweet):
        return [t.encode("utf8") for t in tweet.split("\t")]

In [44]:
encode_tweet(frist_tweet)

['en',
 '345963923251539968',
 'Sat Jun 15 18:00:01 +0000 2013',
 'Letataleta',
 'RT @silsilfani: the world is not a wish-granting machine. dont be surprised when everything always end up disappointing.']

Later on using the filter function as done below we will select only a useful subset of the data

In [ ]:
"""remove urls"""

#TODO

In [ ]:
"""remove emojis"""

#TODO

In [ ]:
"""remove punctuation"""

#TODO

In [ ]:
"""Lemmatization"""

#TODO

In [45]:
"""select a subset of the data"""

data = data.filter(selection_tweet)

en_data = data.filter(lambda x : x[:2]=='en')
es_data = data.filter(lambda x : x[:2]=='es')
fr_data = data.filter(lambda x : x[:2]=='fr')

data_2012 = data.filter(lambda tweet : 
                        encode_tweet(tweet)[2][-4:] == '2012')
data_2013 = data.filter(lambda tweet : 
                        encode_tweet(tweet)[2][-4:] == '2013')


In [46]:
some_fr_tweets = fr_data.take(5)
some_fr_tweets

[u'fr\t345963923255730176\tSat Jun 15 18:00:01 +0000 2013\t_irem61_\tRT @DHC_Music: Terrorism ... #FreePalestina http://t.co/OLWnVlW682',
 u'fr\t345963923335413761\tSat Jun 15 18:00:01 +0000 2013\tHairCutGroup\tHair by Unihair-boutique\\nhttp://t.co/fRGYMrRcQe http://t.co/7sXXzn1x34',
 u'fr\t345963923465441280\tSat Jun 15 18:00:01 +0000 2013\tChajopico\tRT @KimberleyNoe: " J\'ai rencontr\xe9 un autre. On a pass\xe9 des moments formidables. C\'\xe9tait bien mais ce n\'\xe9tait pas toi. "',
 u'fr\t345963923360604161\tSat Jun 15 18:00:01 +0000 2013\tyousinceforever\tDu coup sa ma blaser de tourner en rond la!',
 u"fr\t345963923398328320\tSat Jun 15 18:00:01 +0000 2013\tGraciaDiamond_\tLa meilleure de #SS7 c'est Ana\xefs sans aucun doute!!! ;-)"]

In [47]:
some_fr_tweets = [encode_tweet(tweet) for tweet in some_fr_tweets]

In [48]:
print 'Some french tweets:'
for ind, t in enumerate(some_fr_tweets):
    print ind + 1,')User name:',t[3]
    print '         Tweets:', t[4]
    print '         at:', t[2]
    print 

Some french tweets:
1 )User name: _irem61_
         Tweets: RT @DHC_Music: Terrorism ... #FreePalestina http://t.co/OLWnVlW682
         at: Sat Jun 15 18:00:01 +0000 2013

2 )User name: HairCutGroup
         Tweets: Hair by Unihair-boutique\nhttp://t.co/fRGYMrRcQe http://t.co/7sXXzn1x34
         at: Sat Jun 15 18:00:01 +0000 2013

3 )User name: Chajopico
         Tweets: RT @KimberleyNoe: " J'ai rencontré un autre. On a passé des moments formidables. C'était bien mais ce n'était pas toi. "
         at: Sat Jun 15 18:00:01 +0000 2013

4 )User name: yousinceforever
         Tweets: Du coup sa ma blaser de tourner en rond la!
         at: Sat Jun 15 18:00:01 +0000 2013

5 )User name: GraciaDiamond_
         Tweets: La meilleure de #SS7 c'est Anaïs sans aucun doute!!! ;-)
         at: Sat Jun 15 18:00:01 +0000 2013



# A very simple Topic Modeling using LDA

In order to familiarize ourselves with the dataset we started with a very simple approach for topic extraction. For achieving this we will use the Latent Dirichlet allocation algorithm. We first need to build the tf-idf matrix using our data and then then pass it as a parameter to the LDA method. We also need to specify the number of topics to be extracted from the dataset, α(parameter of the Dirichlet prior on the per-document topic distributions) and β(parameter of the Dirichlet prior on the per-topic word distribution). We will determine this values in the next milestone. 

In [28]:
"""Take only the data having all 5 fields and in english"""
en_data = data.filter(lambda x : x[:2]=='en')
en_data = en_data.filter(selection_tweet)

"""Encode UTF-8"""
en_data = en_data.map(encode_tweet)

"""Take only ID and CONTENT of a tweet"""
tweets = en_data.map(lambda tweet : Row(id=tweet[1], sentence=tweet[4]))

"""Create DF"""
df_tweets = sqlContext.createDataFrame(tweets)

df_tweets.show(3)

+------------------+--------------------+
|                id|            sentence|
+------------------+--------------------+
|345963923251539968|RT @silsilfani: t...|
|345963923297673217|RT @WhosThisHoe: ...|
|345963923259924480|Can't stand peopl...|
+------------------+--------------------+
only showing top 3 rows



In [30]:
"""Tokenization"""
regexTokenizer = RegexTokenizer(inputCol="sentence", outputCol="raw", pattern="\\W")
regexTokenized = regexTokenizer.transform(df_tweets)

regexTokenized.show(3)

+------------------+--------------------+--------------------+
|                id|            sentence|                 raw|
+------------------+--------------------+--------------------+
|345963923251539968|RT @silsilfani: t...|[rt, silsilfani, ...|
|345963923297673217|RT @WhosThisHoe: ...|[rt, whosthishoe,...|
|345963923259924480|Can't stand peopl...|[can, t, stand, p...|
+------------------+--------------------+--------------------+
only showing top 3 rows



In [31]:
"""Remove Stop-words"""
remover = StopWordsRemover(inputCol="raw", outputCol="filtered")
removed_stopwords = remover.transform(regexTokenized)

removed_stopwords.show(3)

+------------------+--------------------+--------------------+--------------------+
|                id|            sentence|                 raw|            filtered|
+------------------+--------------------+--------------------+--------------------+
|345963923251539968|RT @silsilfani: t...|[rt, silsilfani, ...|[rt, silsilfani, ...|
|345963923297673217|RT @WhosThisHoe: ...|[rt, whosthishoe,...|[rt, whosthishoe,...|
|345963923259924480|Can't stand peopl...|[can, t, stand, p...|[t, stand, people...|
+------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [21]:
"""apply lemmatization and remove punctuation"""


'Lemmatization'

 Warning: Computation of TF-IDF and LDA take a lot of time (1h30 on the cluster). This is the reason why we have an extra python file(helloworld.py) to submit to the cluster for the computation.

In [ ]:
"""TF-IDF"""

cv = CountVectorizer(inputCol="filtered", outputCol="vectors")
count_vectorizer_model = cv.fit(removed_stopwords)
tf = count_vectorizer_model.transform(removed_stopwords)

idf = IDF(inputCol="vectors", outputCol="tfidf")
idfModel = idf.fit(tf)
tfidf = idfModel.transform(tf)

tfidf.show(3)

In [ ]:
"""Topics extraction with LDA"""

nbTopics=100
n_terms=15

corpus = tfidf.select(F.col('id').cast("long"), 'tfidf').rdd.map(lambda x: [x[0], x[1]])
ldaModel = LDA.train(corpus, k=nbTopics)


topics = ldaModel.describeTopics(maxTermsPerTopic=n_terms)
vocabulary = count_vectorizer_model.vocabulary

"""Store result"""
with open("topics.pickle", "wb") as f:
    pickle.dump(topics, f)
with open("vocabulary.pickle", "wb") as f:
    pickle.dump(vocabulary, f)   

In [ ]:
"""Load result computed from cluster"""

with open("topics.pickle", "rb") as f:
    topics = pickle.load(f)
    
with open("vocabulary.pickle", "rb") as f:
    vocabulary = pickle.load(f)

In [ ]:
for topic in range(len(topics)):
    print("topic {} : ".format(topic))
    words = topics[topic][0]
    scores = topics[topic][1]
    for word in range(len(words)):
        print(vocabulary[words[word]], "-", scores[word])

# Example of the pipeline that we will follow for the LDA algorithm

In [20]:
# random dataframe 
sentenceDataFrame = sqlContext.createDataFrame([
    (0, "Hi I heard about Spark"),
    (1, "I wish Java could use case classes"),
    (2, "Logistic,regression,models,are,neat"),
    (3, "I want a coffee before going to bed "),
    (4, "Today is a big day !!!")
], ["id", "sentence"])

# tokenization
regexTokenizer = RegexTokenizer(inputCol="sentence", outputCol="words", pattern="\\W")
regexTokenized = regexTokenizer.transform(sentenceDataFrame)

# remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
removed = remover.transform(regexTokenized)

# create the tf-idf matrix
cv = CountVectorizer(inputCol="filtered", outputCol="vectors")
count_vectorizer_model = cv.fit(removed)
tf = count_vectorizer_model.transform(removed)

idf = IDF(inputCol="vectors", outputCol="tfidf")
idfModel = idf.fit(tf)
tfidf = idfModel.transform(tf)

# initialize parameters
nbTopics=3
n_terms=3

corpus = tfidf.select(F.col('id').cast("long"), 'tfidf').rdd.map(lambda x: [x[0], x[1]])
ldaModel = LDA.train(corpus, k=nbTopics)

# extracting topics
topics = ldaModel.describeTopics(maxTermsPerTopic=n_terms)

# extraction vocabulary
vocabulary = count_vectorizer_model.vocabulary
file = open("testfile.txt","w")
for topic in range(len(topics)):
    print("topic {} : ".format(topic))
    file.write("topic {} : \n".format(topic)) 
    words = topics[topic][0]
    scores = topics[topic][1]
    for word in range(len(words)):
        file.write("{} - {}\n".format(vocabulary[words[word]], scores[word]))
        print(vocabulary[words[word]], "-", scores[word])
file.close()

DataFrame[id: bigint, sentence: string]
topic 0 : 
(u'regression', '-', 0.05145751141777857)
(u'neat', '-', 0.05091900278065432)
(u'logistic', '-', 0.05087718877476008)
topic 1 : 
(u'coffee', '-', 0.0508638048864802)
(u'case', '-', 0.05080822565440008)
(u'want', '-', 0.05067566719691306)
topic 2 : 
(u'hash', '-', 0.05181265926459157)
(u'today', '-', 0.051378772846123556)
(u'day', '-', 0.051329188339834325)


# Milestone 3: the data story

In [ ]:
"""Play around with the parameters of the LDA algorithm in order to find the optimal values for α and β. """

#TODO

In [ ]:
"""repeat the same technique for spanish and french"""

#TODO

In [ ]:
"""algorithm for detecting the top trends"""

#TODO

In [ ]:
"""trying to find patterns between trends"""

#TODO

In [ ]:
"""analyse the data per user 
    ex. which topics he tweets most about ? 
        does it change over time ? """

#TODO

In [ ]:
"""find top users that were the most mentioned by somebody else"""

#TODO

In [ ]:
"""visualization of the result(per language, per month ...)"""

#TODO